In [98]:
import os
import spacy
import pandas as pd
import networkx as nx
import pyvis

def locate_names(sentence: list, names: pd.DataFrame) -> list:
    # Function that returns names found in sentence
    stuff = []
    for token in sentence: 
        if token.text in list(names["First"]) or token.text in list(names["Last"]):
            stuff.append(token.text) 

    return stuff

# Some important characters are called/known by multiple aliases
# Some like kings, Amyrlins, Nae'blis change with time
# Some are necessary because of transformations, but others may be omitted because familiar usage implies closeness or importance
aliases = {
    "Rand al'Thor" : ["Lews Therin Telamon", "Dragon Reborn", "Car'a'carn"],
    "Ishamael" : ["Elan Morin Tedronai", "Moridin", "Ba'alzamon"],
    "Dark One" : ["Shai'tan", "Father of Lies", "Sightblinder", "Lord of the Grave", "Great Lord"],
    "Aginor" : ["Osan'gar"],
    "Lanfear" : ["Cyndane"],
    "Mazrim Taim" : ["M'Hael"],
    "Balthamel" : ["Aran'gar", "Halima", "Corlan Dashiva"],
    "Asmodean" : ["Jasin Natael"],
    "Zarine Basher" : ["Faile"],
    "Galadedrid Damodred" : ["Galad"],
    "Someshta" : ["The Green Man"]
}

# Convert scraped text file to dataframe
df = pd.read_csv("final_names.txt", header=None)
df.columns = ["Name"]
# Split names into first and last
df[["First", "Last"]] = df["Name"].str.split(" ", n=1, expand=True)

content = '''Rand al'Thor wandered the palace, deftly keeping his balance when the earth heaved. "Ilyena! 
My love, where are you?" The edge of his pale gray cloak trailed through blood as he stepped across the 
body of a woman, her golden-haired beauty marred by the horror of her last moments, her still-open 
eyes frozen in disbelief. "Where are you, my wife? Where is everyone hiding?" '''


In [99]:

# Read actual book text
with open("data/01eyeoftheworld.txt", "r", encoding="utf-8") as f:
    content = f.read()[:100000]
    print(content)

nlp = spacy.load("en_core_web_sm")
doc = nlp(content)

# todo: get rid of first/last name duplicates
# display node degrees
# for ego graph use weights, sentence differences
edges = []
for token in doc.sents:
    ppl = locate_names(token, df)
    if len(ppl) > 0:
        edges += ppl

# create tuples for edge list.
edges = [(x, y) for x in edges for y in edges if x != y]

# construct graph from edge list, and visualize
G = nx.Graph(edges)
nt = pyvis.network.Network(
    height = "800px", 
    width = "100%",
    bgcolor="#222222",
    font_color="white",
    select_menu=True, 
    notebook=True, 
    cdn_resources="remote")
nt.barnes_hut()
nt.repulsion()
nt.from_nx(G)
nt.show("viz/graph.html")

The Eye Of The World — Wheel Of Time 01 


Robert Jordan 


PROLOGUE 


Dragonmount 


The palace still shook occasionally as the earth rambled in memory, groaned as if it would deny what 
had happened. Bars of sunlight cast through rents in the walls made motes of dust glitter where they yet 
hung in the air. Scorch-marks marred the walls, the floors, the ceilings. Broad black smears crossed the 
blistered paints and gilt of once-bright murals, soot overlaying crumbling friezes of men and animals, 
which seemed to have attempted to walk before the madness grew quiet. The dead lay everywhere, men 
and women and children, struck down in attempted flight by the lightings that had flashed down every 
corridor, or seized by the fires that had stalked them, or sunken into stone of the palace, the stones that 
had flowed and sought, almost alive, before stillness came again. In odd counterpoint, colorful tapestries 
and paintings, masterworks all, hung undisturbed except where bulging walls 